## Information

Alternative libaries:

https://github.com/studioimaginaire/phue (using this one)

https://github.com/issackelly/python-hue

https://github.com/sontek/bulby

Executing the notebook:

https://nbconvert.readthedocs.io/en/latest/execute_api.html

In [9]:
from phue import Bridge
from datetime import datetime
import dateutil.parser
import time

In [10]:
bridge = Bridge('192.168.1.2')
bridge.connect()

In [11]:
# Protocol

# { "9:30 - 10:00" : taskObjectCallable }

def InterpretSchedule(entry: str) -> "('room', start time, end time)":
    """String specification to meaningful tuple.
    
    Input examples:
      "10:00 - 11:00" -> (start time, end time)
      "10:00" -> (start time, end of day)
    """
    def process(x):
        return dateutil.parser.parse(x)
    times = [process(x.strip()) for x in entry.split('-')]
    if len(times) == 1:
        return (times[0], dateutil.parser.parse("23:59"))
    else:
        return (times[0], times[1])


def ProcessScheduleEntry(entry, task):
    start, end = InterpretSchedule(entry)
    return (start, end, task)


def FormQueue(entries):
    queue = [ProcessScheduleEntry(k, v) for k, v in entries]
    return sorted(queue)
    

In [12]:
# Flow Control

def ProcessTasks(taskQueue, api, bridge):
    i = 0
    while i < len(taskQueue):
        start_t, end_t, task = taskQueue[i]
        current_t = datetime.now()
        if current_t > end_t: # missed it
            print("Task deleted: ", task, "at", datetime.now().isoformat())
            del taskQueue[i]
            continue;
        if current_t >= start_t: # time for the task
            if task(api, bridge):
                print("Task complete: ", task, "at", datetime.now().isoformat())
                del taskQueue[i]
                continue;
            else: # not successful leave in the queue
                #print("could not finish task", task)
                i = i + 1
                continue;
        if current_t < start_t: # sorted list, so no need to dive deeper
            break;
            
            
            

In [33]:
# Light control methods

def GetLightId(api, name) -> int:
    for _id, light in api['lights'].items():
        if light['name'] == name:
            return int(_id)
    raise Exception("Light not foud with name: "+ name)

    
def GetLightAPI(api, name):
    return api['lights'][str(GetLightId(api, name))]
    
    
def IsLightOn(api, name) -> bool:
    return GetLightAPI(api, name)['state']['on']


def SetColor(bridge, name, color, transitiontime):
    "Color is maped from 0 (red) to 1 (blue)."
    assert color >= 0.0 and color <= 1.0
    _min, _max = sorted(GetLightAPI(bridge.get_api(), name)['capabilities']['control']['ct'].values())
    color_hue_notation = int(_max - color*(_max-_min))
    bridge.set_light(name, 'ct', color_hue_notation, transitiontime=transitiontime)
    
    
def HueColor(color):
    "Hardcoded color translation."
    return  int(454 - color*(454-153))
    
    
def SetBrightness(bridge, name, brightness, transitiontime):
    assert brightness >= 0.0 and brightness <= 1.0 
    brightness_hue_notation = int(brightness*(255-0))
    bridge.set_light(name, 'bri', brightness_hue_notation, transitiontime=transitiontime)
    

def SetPower(bridge, name, on: bool):
    light_id = GetLightId(bridge.get_api(), name)
    bridge.set_light(light_id, 'on', on, transitiontime=0)

    
class Lights:
    """Light transition class - for ambience setting."""
    def __init__(self, name, color=None, brightness=None, transitiontime=0):
        self.name = name
        self.color = color
        self.brightness = brightness
        self.transitiontime = int(transitiontime * 10) # Hue takes decisec, we want seconds
        
    def __call__(self, api, bridge):
        if not IsLightOn(api, self.name): 
            return False
        command = {'transitiontime': self.transitiontime, 'on': True}
        if self.color is not None:
            command['ct'] = HueColor(self.color)
        if self.brightness is not None:
            brightness_hue_notation = int(self.brightness*(255-0))
            command['bri'] = brightness_hue_notation
        bridge.set_light(self.name, command)
        return True
    
    def __repr__(self):
        return "Lights("+self.name+", color="+str(self.color)+", brightness="+str(self.brightness)+")"
    def __lt__(self, other):
         return str(self) < str(other)

    
class WakeupLights:
    """Wakeup morning routine for thr lights.
    
    Slowly increase the brightness in the morning to wake up the user. 
    """
    def __init__(self, name, transitiontime):
        self.name = name
        self.transitiontime = int(transitiontime * 10) # Hue takes decisec, we want seconds
        
    def __call__(self, api, bridge):
        if IsLightOn(api, self.name): 
            # only do something if the light is off
            return True
        light_id = GetLightId(api, self.name)
        # direct access to reduce time lag between calls
        command = {'transitiontime' : 0, 'on' : True, 'bri' : 0, 'ct': HueColor(0.8)}
        bridge.set_light(self.name, command)
        # wake up slowly
        SetBrightness(bridge, self.name, 1.0, self.transitiontime)
        return True
    
    def __repr__(self):
        return "WakeupLights("+self.name+")"
    def __lt__(self, other):
         return str(self) < str(other)



In [41]:
def Weekend():
    return [
        # morning
        ["07:00 - 20:00", Lights("Living room", color=0.8, brightness=1.0, transitiontime=10)],
        ["07:00 - 20:00", Lights("Hall way", color=0.8, brightness=1.0, transitiontime=10)],
        ["08:00 - 20:00", Lights("Bedroom", color=0.8, brightness=1.0, transitiontime=10)],
        # evening
        ["20:00 - 22:00", Lights("Living room", color=0.3,  transitiontime=60)],
        ["20:00 - 22:00", Lights("Hall way", color=0.3,  transitiontime=60)],
        ["20:00 - 22:00", Lights("Bedroom", color=0.3,  transitiontime=60)],
        # late evening
        ["22:00 - 23:30", Lights("Living room", color=0.0, brightness=0.8, transitiontime=2*60)],
        ["22:00 - 23:30", Lights("Hall way", color=0.0, brightness=0.8, transitiontime=2*60)],
        ["22:00 - 23:30", Lights("Bedroom", color=0.0, brightness=0.8, transitiontime=2*60)],
        # night
        ["23:30 - 23:59", Lights("Living room", color=0.0, brightness=0.5, transitiontime=2*60)],
        ["23:30 - 23:59", Lights("Hall way", color=0.0, brightness=0.5, transitiontime=2*60)],
        ["23:30 - 23:59", Lights("Bedroom", color=0.0, brightness=0.5, transitiontime=2*60)],
    ]


def Weekday():
    return [
        ["07:00 - 08:00", WakeupLights("Bedroom", transitiontime=int(15*60))], 
    ] + Weekend()


In [35]:
def IsNewDay() -> bool:
    return datetime.now().hour < 2

In [42]:
# Main

queue = []
i = 0

while(True):
    if not queue and (IsNewDay() or i==0): # add time check
        is_weekend = datetime.today().weekday() >= 5
        if is_weekend:
            queue = FormQueue(Weekend())
            print("Filled the queue with Weekend() at", datetime.now().isoformat())
        else:
            queue = FormQueue(Weekday())
            print("Filled the queue with Weekday() at", datetime.now().isoformat())
    try:
        api = bridge.get_api()
        ProcessTasks(queue, api, bridge)
    except:
        print("An exception occurred on ", datetime.now().isoformat()) 
     
    i = i + 1
    time.sleep(5)
    

Filled the queue with Weekday() at 2019-09-09T21:56:07.212351
Task deleted:  WakeupLights(Bedroom) at 2019-09-09T21:56:07.292431
Task deleted:  Lights(Hall way, color=0.8, brightness=1.0) at 2019-09-09T21:56:07.294006
Task deleted:  Lights(Living room, color=0.8, brightness=1.0) at 2019-09-09T21:56:07.294488
Task deleted:  Lights(Bedroom, color=0.8, brightness=1.0) at 2019-09-09T21:56:07.294732
Task complete:  Lights(Bedroom, color=0.3, brightness=None) at 2019-09-09T21:56:07.327093
Task complete:  Lights(Hall way, color=0.3, brightness=None) at 2019-09-09T21:56:07.389447
Task complete:  Lights(Living room, color=0.3, brightness=None) at 2019-09-09T21:56:07.445701
Task complete:  Lights(Bedroom, color=0.0, brightness=0.8) at 2019-09-09T22:00:02.054404
Task complete:  Lights(Hall way, color=0.0, brightness=0.8) at 2019-09-09T22:00:02.120086
Task complete:  Lights(Living room, color=0.0, brightness=0.8) at 2019-09-09T22:00:02.161721


OSError: [Errno 51] Network is unreachable